In [1]:
# import pacakges
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns',None)

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

import pickle

# Import Data

In [2]:
X_train = pd.read_csv('data/train-test/train_nulls_dropped.csv')
X_test = pd.read_csv('data/train-test/test_nulls_dropped.csv')

In [3]:
features = ['Pitcher_name','batter_name','pitch_name','release_speed', 'release_pos_x', 'release_pos_z', 'stand',
       'p_throws', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z','inning', 'inning_topbot', 'effective_speed',
       'release_spin_rate', 'pitch_number', 'bat_score', 'fld_score', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'outs_when_up']
features2 = ['batter_name','pitch_name','release_speed', 'release_pos_x', 'release_pos_z', 'stand',
       'p_throws', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z','inning', 'inning_topbot', 'effective_speed',
       'release_spin_rate', 'pitch_number', 'bat_score', 'fld_score', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'outs_when_up']
features3 = ['pitch_name','release_speed', 'release_pos_x', 'release_pos_z', 'stand',
       'p_throws', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z','inning', 'inning_topbot', 'effective_speed',
       'release_spin_rate', 'pitch_number', 'bat_score', 'fld_score', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'outs_when_up']

In [4]:
X_train = X_train[features3]
X_test = X_test[features3]

In [32]:
X_train.columns[:27]

Index(['pitch_name', 'release_speed', 'release_pos_x', 'release_pos_z',
       'stand', 'p_throws', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x',
       'plate_z', 'inning', 'inning_topbot', 'effective_speed',
       'release_spin_rate', 'pitch_number', 'bat_score', 'fld_score', 'vx0',
       'vy0', 'vz0', 'ax', 'ay', 'az', 'outs_when_up'],
      dtype='object')

In [5]:
# We will pd.get_dummies 
X_train = pd.get_dummies(X_train, columns=['pitch_name'])
X_test = pd.get_dummies(X_test, columns=['pitch_name'])

In [6]:
# drop columns that don't match in each other
drop_test = [col for col in X_test.columns if col not in X_train.columns]
drop_train = [col for col in X_train.columns if col not in X_test.columns]

X_train.drop(columns=drop_train, inplace=True)
X_test.drop(columns=drop_test, inplace=True)

In [7]:
X_train.columns == X_test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [8]:
y_train = pickle.load(open('assets/y_train.pkl','rb'))
y_test = pickle.load(open('assets/y_test.pkl','rb'))

In [10]:
y_train.head()

0    strike
1    strike
2    strike
3      ball
4    strike
Name: strike_attempt, dtype: object

# Set up Pipeline/GridSearchCV

---
Unable to do GridSearch because I have been running into too many memory errors

In [11]:
# list(range(1, 101, 10))

In [12]:
# rf_params = {
#     'n_neighbors': list(range(1, 101, 10)),
#     'min_samples_split':[2,3],
#     'max_features':["auto",28],
#     'max_samples':[.3,.5,.7]
# } 

In [13]:
# rf = GridSearchCV(RandomForestClassifier(random_state=66),
#                   param_grid=rf_params,
#                   cv=5,
#                   n_jobs=8,
#                   verbose = 2)

In [14]:
# rf.fit(X_train,y_train)

In [15]:
rf = RandomForestClassifier(random_state=66, n_jobs=8)
rf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=8, random_state=66)

In [17]:
preds = rf.predict(X_test)

In [18]:
accuracy_score(y_test, preds)

0.7092507858764223

In [21]:
df = pd.DataFrame(rf.feature_importances_, index=X_train.columns)

In [27]:
df.sort_values(0, ascending=False).head(35)

,0
plate_x,0.137504
plate_z,0.115888
vz0,0.064031
vx0,0.051252
release_pos_x,0.038365
az,0.037864
release_pos_z,0.036782
ax,0.036452
ay,0.035946
release_spin_rate,0.035828


In [28]:
pickle.dump(rf, open('assets/rf_no_pitcher_12062020.pkl','wb'))

# Random Forest model, no pitchers and batters

In [9]:
rf_params = {
#     'n_neighbors': list(range(1, 101, 10)),
    'min_samples_split':[2,3],
#     'min_samples_leaf':[1,2,3],
    'max_features':['auto','log2'],
    'max_samples':[.95,.99]
} 
rf2 = GridSearchCV(RandomForestClassifier(random_state=66),
                  param_grid=rf_params,
                  cv=5,
                  n_jobs=8,
                  verbose = 2)

In [ ]:
rf2.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
